In [87]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import strategy_vgg_pretrained_model; reload(strategy_vgg_pretrained_model)
from keras.models import Sequential
import file_utils; reload(file_utils)
from keras.optimizers import RMSprop
from keras.utils.np_utils import to_categorical

In [95]:
% rm -rf ../data/kaggle_fisheries/output
% mkdir ../data/kaggle_fisheries/output
% cp -R  ../data/kaggle_fisheries/train ../data/kaggle_fisheries/output/train
%run createSample.py

## Strategy 1 : Train all the last layers of VGG Model

In [43]:
num = 64
path = "../data/kaggle_fisheries/output/sample/"
rootPath = "../data/kaggle_fisheries/output/"
batchGenerator_train = file_utils.fileBatchGenerator(rootPath+"train/", num)
batchGenerator_valid = file_utils.fileBatchGenerator(rootPath+"valid/", num)

Found 2395 images belonging to 8 classes.
Found 261 images belonging to 8 classes.


### Get The convolutional Features for faster training

In [45]:
vggModel = strategy_vgg_pretrained_model.createVGGModelAllBlock()

In [46]:
vggModelConvolutionBlock = strategy_vgg_pretrained_model.getConvModelFromFullVGGModel(vggModel)
conv_features_valid = vggModelConvolutionBlock.predict_generator(batchGenerator_valid, batchGenerator_valid.nb_sample )
conv_features_train= vggModelConvolutionBlock.predict_generator(batchGenerator_train, batchGenerator_train.nb_sample )

<class 'keras.layers.core.Dense'>
<class 'keras.layers.core.Dropout'>
<class 'keras.layers.core.Dense'>
<class 'keras.layers.core.Dropout'>
<class 'keras.layers.core.Dense'>
<class 'keras.layers.core.Flatten'>
<class 'keras.layers.pooling.MaxPooling2D'>
<class 'keras.layers.convolutional.Convolution2D'>
convolution block


In [47]:
file_utils.saveArray(path +"conv_features_valid", conv_features_valid)
file_utils.saveArray(path +"conv_features_train", conv_features_train)

### Setting up the Dense Layers for Training 

In [90]:
conv_features_valid = file_utils.loadArray(path +"conv_features_valid")
conv_features_train = file_utils.loadArray(path +"conv_features_train")
#strategy_vgg_pretrained_model.markLayersAsTrainable(vggModelDenseBlock.layers, True)
opt = RMSprop(lr=0.001, rho=0.7)
one_hot_encoded_labels_validation = to_categorical(batchGenerator_valid.classes)
one_hot_encoded_labels_training = to_categorical(batchGenerator_train.classes)

### When all the dense layers are set to Trainable

In [93]:
vggModelDenseBlock = strategy_vgg_pretrained_model.getFCModelFromFullVGGModel(vggModel)
strategy_vgg_pretrained_model.modifyAnyModelFor8Output(vggModelDenseBlock, True)
vggModelDenseBlock.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
vggModelDenseBlock.fit(conv_features_train, one_hot_encoded_labels_training, batch_size = 64, nb_epoch=10, validation_data=(conv_features_valid, one_hot_encoded_labels_validation) )

Train on 2395 samples, validate on 261 samples
Epoch 1/10
2395/2395 [==============================] - 4s - loss: 8.8922 - acc: 0.4459 - val_loss: 8.7075 - val_acc: 0.4598 128/2395 [>.............................] - ETA: 4s - loss: 10.4895 - acc: 0.3047

### When all but last the dense layers are set to NOT Trainable

In [92]:
vggModelDenseBlock = strategy_vgg_pretrained_model.getFCModelFromFullVGGModel(vggModel)
strategy_vgg_pretrained_model.modifyAnyModelFor8Output(vggModelDenseBlock, False)
vggModelDenseBlock.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
vggModelDenseBlock.fit(conv_features_train, one_hot_encoded_labels_training, batch_size = 64, nb_epoch=10, validation_data=(conv_features_valid, one_hot_encoded_labels_validation) )

Train on 2395 samples, validate on 261 samples
Epoch 1/10
2395/2395 [==============================] - 1s - loss: 8.9387 - acc: 0.4276 - val_loss: 8.3674 - val_acc: 0.4559 128/2395 [>.............................] - ETA: 1s - loss: 11.2116 - acc: 0.2422